# Реализация CART

### Принцип работы дерева решений (CART)
1) создаётся корневой узел на основе наилучшего разбиения;
2) тренировочный набор разбивается на 2 поднабора: всё что соответствует условию разбиения отправляется в левый узел, остальное — в правый;
3) далее рекурсивно для каждого тренировочного поднабора повторяются шаги 1-2 пока не будет достигнут один из основных критериев останова: максимальная глубина, максимальное количество листьев, минимальное количество наблюдений в листе или минимальное снижение загрязнения в узле.

In [3]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=3, regression=False):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.regression = regression
        self.tree = None
        
        if self.regression:
            self.impurity_func = lambda y: np.mean((y - np.mean(y))**2)
        else:
            self.impurity_func = lambda y: 1 - np.sum((np.bincount(y) / len(y))**2)

    def _find_best_split(self, X, y):
        best_g = float('inf')  # Минимальное значение примеси
        best_j, best_t = None, None  # Лучший признак и порог
        
        # Перебор всех признаков
        for j in range(X.shape[1]):
            thresholds = np.unique(X[:, j])  # Уникальные значения признака
            for t in thresholds:
                left_mask = X[:, j] <= t
                n_left = np.sum(left_mask)
                n_right = len(y) - n_left
                
                # Проверка минимального размера ветвей
                if n_left < self.min_samples_split or n_right < self.min_samples_split:
                    continue
                
                # Взвешенная примесь
                g = (n_left/len(y))*self.impurity_func(y[left_mask]) + (n_right/len(y))*self.impurity_func(y[~left_mask])
                
                if g < best_g:
                    best_g, best_j, best_t = g, j, t
        return best_j, best_t

    def _build_tree(self, X, y, depth=0):
        """Рекурсивное построение дерева"""
        if (depth >= self.max_depth or len(y) < self.min_samples_split or (not self.regression and len(np.unique(y)) == 1)):
            return self._leaf_value(y)
        
        j, t = self._find_best_split(X, y)
        if j is None:
            return self._leaf_value(y)
        
        left_mask = X[:, j] <= t
        return {
            'feature': j,
            'threshold': t,
            'left': self._build_tree(X[left_mask], y[left_mask], depth + 1),
            'right': self._build_tree(X[~left_mask], y[~left_mask], depth + 1)
        }

    def _leaf_value(self, y):
        """Возвращает значение листа: среднее для регрессии или моду для классификации."""
        return np.mean(y) if self.regression else np.argmax(np.bincount(y))

    def fit(self, X, y):
        """Обучение модели"""
        self.tree = self._build_tree(np.array(X), np.array(y))

    def _predict_sample(self, x, node):
        if isinstance(node, dict):  # Если узел не лист
            if x[node['feature']] <= node['threshold']:
                return self._predict_sample(x, node['left'])
            else:
                return self._predict_sample(x, node['right'])
        return node  # Возврат значения листа

    def predict(self, X):
        """Прогноз для набора данных."""
        X = np.array(X)
        return np.array([self._predict_sample(x, self.tree) for x in X])


In [4]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, 
    precision_score, cohen_kappa_score, matthews_corrcoef, 
    mean_absolute_error, mean_squared_error, r2_score, 
    mean_squared_log_error, mean_absolute_percentage_error
)
from sklearn.model_selection import train_test_split

# Классификация
df_class = pd.read_csv(r'C:\Users\Zver\Desktop\ML\data\smoke_detector_task_filtered.csv', 
                         sep=',', encoding='utf-8')
df_class = df_class.drop(['Unnamed: 0'], axis=1)
class_label = 'Fire Alarm'
class_features = ['TVOC[ppb]', 'eCO2[ppm]', 'Temperature[C]', 'PM2.5', 'NC2.5', 
                  'Humidity[%]', 'Raw H2', 'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 
                  'NC0.5', 'NC1.0']
X_class, y_class = df_class[class_features].values, df_class[class_label].values

X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42)

clf = DecisionTree(max_depth=3, regression=False)
clf.fit(X_class_train, y_class_train)
preds_class = clf.predict(X_class_test)

acs = accuracy_score(y_class_test, preds_class)
rs = recall_score(y_class_test, preds_class)
ps = precision_score(y_class_test, preds_class)
f1 = f1_score(y_class_test, preds_class)
ch = cohen_kappa_score(y_class_test, preds_class)
mcc = matthews_corrcoef(y_class_test, preds_class)
print("Классификация:", acs, rs, ps, f1, ch, mcc)



# Регрессия
df_reg = pd.read_csv(r'C:\Users\Zver\Desktop\ML\data\moldova_cars_task_filtered_ohe.csv', 
                     sep=',', encoding='utf-8')
reg_label = 'Price(euro)'
reg_features = ['Year', 'Distance', 'Engine_capacity(cm3)', 'Transmission']
X_reg, y_reg = df_reg[reg_features].values, df_reg[reg_label].values

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42)

reg = DecisionTree(regression=True, max_depth=5)
reg.fit(X_reg_train, y_reg_train)
preds_reg = reg.predict(X_reg_test)

mae = mean_absolute_error(y_reg_test, preds_reg)
r2 = r2_score(y_reg_test, preds_reg)
mse = mean_squared_error(y_reg_test, preds_reg)
rmse = np.sqrt(mse)
rmsle = np.sqrt(mean_squared_log_error(y_reg_test, preds_reg))
mape = mean_absolute_percentage_error(y_reg_test, preds_reg)
print("Регрессия:", mae, r2, mse, rmse, rmsle, mape)

Классификация: 0.9822979291917168 0.9998836668217775 0.9760390642743584 0.987817492242271 0.9554683093156721 0.9563992583172939
Регрессия: 3079.1279165267824 0.7154785009027032 27354750.765006647 5230.176934388229 0.36175129090153424 0.3144852366500766
